In [1]:
import tensorflow as tf
import numpy as np
import datetime, time

In [2]:
def genObs(length, full_length, dict_size=3):
    base = np.random.choice(range(dict_size), size=length)
    pad_length = full_length - length
    return np.concatenate([1 + base, np.zeros(pad_length)])

def getTarget(x):
    s = np.argmin(x)
    return np.concatenate([np.array(list(reversed(x[:s]))), x[s:]])

def genSample(num, length=20):
    x = [genObs(np.random.randint(length), full_length=length) for _ in range(num)]
    y = [getTarget(y) for y in x]
    return np.array(x), np.array(y)

def randomBatch(tensorTuple, batchSize=64):
    ids = np.random.choice(range(tensorTuple[0].shape[0]), batchSize)
    return (x[ids,] for x in tensorTuple)

def shuffleBatches(tensorTuple, batchSize=64):
    if type(tensorTuple) is list or type(tensorTuple) is tuple: 
        ids = list(range(tensorTuple[0].shape[0]))
        np.random.shuffle(ids)
        for i in range(0,len(ids),batchSize):
            lst = min(len(ids), i + batchSize)
            yield (np.array(x[ids[i:lst],]) for x in tensorTuple)
    else:
        ids = list(range(tensorTuple.shape[0]))
        np.random.shuffle(ids)
        for i in range(0,len(ids),batchSize):
            lst = min(len(ids), i + batchSize)
            yield np.array(tensorTuple[ids[i:lst],])

In [3]:
train_x, train_y = genSample(100000, length=6)
valid_x, valid_y = genSample(10000, length=6)
valid_x[:10], valid_y[:10]

(array([[ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 3.,  2.,  1.,  1.,  2.,  0.],
        [ 3.,  0.,  0.,  0.,  0.,  0.],
        [ 3.,  1.,  2.,  0.,  0.,  0.],
        [ 3.,  2.,  1.,  3.,  0.,  0.],
        [ 2.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  3.,  1.,  3.,  3.,  0.],
        [ 1.,  1.,  2.,  3.,  0.,  0.],
        [ 3.,  2.,  0.,  0.,  0.,  0.],
        [ 1.,  3.,  0.,  0.,  0.,  0.]]),
 array([[ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 2.,  1.,  1.,  2.,  3.,  0.],
        [ 3.,  0.,  0.,  0.,  0.,  0.],
        [ 2.,  1.,  3.,  0.,  0.,  0.],
        [ 3.,  1.,  2.,  3.,  0.,  0.],
        [ 2.,  0.,  0.,  0.,  0.,  0.],
        [ 3.,  3.,  1.,  3.,  1.,  0.],
        [ 3.,  2.,  1.,  1.,  0.,  0.],
        [ 2.,  3.,  0.,  0.,  0.,  0.],
        [ 3.,  1.,  0.,  0.,  0.,  0.]]))

In [ ]:
SEQ_LEN = 6
DICT_SIZE = 3
ENCODER_RNN_SIZE = [15]
DECODER_RNN_SIZE = [15]
HIDDEN_LAYER = 30

EncoderCell = lambda n: tf.nn.rnn_cell.GRUCell(num_units=n, activation=tf.nn.elu)
DecoderCell = lambda n: tf.nn.rnn_cell.GRUCell(num_units=n, activation=tf.nn.elu)

tf.reset_default_graph()

rnnEncoderCell = tf.nn.rnn_cell.MultiRNNCell([EncoderCell(s) for s in ENCODER_RNN_SIZE], state_is_tuple=True)
rnnDecoderCell = tf.nn.rnn_cell.MultiRNNCell([DecoderCell(s) for s in DECODER_RNN_SIZE], state_is_tuple=True)

tfi_x = tf.placeholder(shape=(None, SEQ_LEN), dtype=tf.int32)
tfi_y = tf.placeholder(shape=(None, SEQ_LEN), dtype=tf.int32)

tfX = tf.one_hot(tfi_x, DICT_SIZE + 1, dtype=tf.float32)
tfY = tf.one_hot(tfi_x, DICT_SIZE + 1, dtype=tf.float32)

_, tfEncodedState0 = tf.nn.dynamic_rnn(rnnCell, inputs=tfX, dtype=tf.float32, time_major=False)
tfEncodedState = tfEncodedState0[-1] #get latest layer in RNN

tfCode0 = tf.layers.dense(tfEncodedState, HIDDEN_LAYER, activation=tf.nn.elu)
 = tf.layers.dense(tfEncodedState, HIDDEN_LAYER, activation=tf.nn.elu)

tfHOut0 = tf.layers.dense(tfH, DICT_SIZE + 1)
tfHOut = tfHOut0[:,:-1,:] #remove last forecast

tfHLoss0 = tf.nn.softmax_cross_entropy_with_logits(labels=tfY, logits=tfHOut) #should be like batch_size x SEQ_LEN-1
tfLoss = tf.reduce_mean(tfHLoss0 * tfM)
tfTrain = tf.train.AdamOptimizer(1e-3).minimize(tfLoss)

tfPredicted = tf.cast(tf.argmax(tfHOut, axis=2), dtype=tf.int32) * tfi_m[:,1:]
tfActual = tfi_x[:,1:] * tfi_m[:,1:]

tfAccuracyChar = tf.reduce_mean(tf.cast(tf.equal(tfActual, tfPredicted), dtype=tf.float32))
tfAccuracyObs = tf.reduce_mean(tf.reduce_min(tf.cast(tf.equal(tfActual, tfPredicted), dtype=tf.float32), axis=1))

tfsLoss = tf.summary.scalar('RMSE', tfLoss)
tfsAccuracy = tf.summary.scalar('Accuracy', 1-tfAccuracyObs)
tfsAll = tf.summary.merge([tfsLoss, tfsAccuracy])
tfsSaver = tf.train.Saver()

tffw = tf.summary.FileWriter('D:/Jupyter/Logs/00_A', tf.get_default_graph())
print('Graph creation complete')